# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re

import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dchen@us.ibm.com/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dchen@us.ibm.com/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
# list(set(df.columns) - set({'id', 'message', 'original', 'genre'}))

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('cleaned_data', engine)
X = df['message'].values
output_columns = list(set(df.columns) - set({'id', 'message', 'original', 'genre'}))
Y = df[output_columns].values

In [4]:
# Y

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """Take the following steps to clean and tokenize text data:
        1. Replace any urls in text with "urlplaceholder" to reduce the data noise caused by url
        2. Split text into tokens.
        3. lemmatize, normalize case, and strip leading and trailing white space.
    Args:
        text (str): the input text to clean and tokenize
    Returns:
        list of string: cleaned and tokenized list of words
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()   
    return [lemmatizer.lemmatize(token).lower().strip() for token in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=20)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
%%time
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

CPU times: user 59.3 s, sys: 193 ms, total: 59.5 s
Wall time: 59.6 s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = pipeline.predict(X_test)

In [9]:
# for columns in output_columns:
for i in range(Y_test.shape[1]):
    print(f"classification_report for category {output_columns[i]}:")
    print(classification_report(Y_test[:,i], Y_pred[:,i]))

classification_report for category fire:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6476
           1       0.50      0.01      0.03        78

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.74      0.51      0.51      6554
weighted avg       0.98      0.99      0.98      6554

classification_report for category medical_products:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6237
           1       0.79      0.05      0.09       317

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.87      0.52      0.53      6554
weighted avg       0.95      0.95      0.93      6554

classification_report for category offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

   micro avg       1.00      1.00      1.00   

/Users/dchen@us.ibm.com/miniconda3/envs/eliza/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
%%time
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__max_features': (None, 5000),
#     'clf__estimator__n_estimators': [50, 100],
#     'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=3).fit(X_train, Y_train)

In [ ]:
cv.best_estimator_.get_params()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
pipeline.predict(['says: west side of Haiti, rest of the country today and tonight'])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
%%time
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=None))])

In [28]:
Y_pred = pipeline.predict(X_test)
for i in range(Y_test.shape[1]):
    print(f"classification_report for category {output_columns[i]}:")
    print(classification_report(Y_test[:,i], Y_pred[:,i]))

classification_report for category military:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5081
           1       0.36      0.36      0.36       163

   micro avg       0.96      0.96      0.96      5244
   macro avg       0.67      0.67      0.67      5244
weighted avg       0.96      0.96      0.96      5244

classification_report for category child_alone:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5244

   micro avg       1.00      1.00      1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

classification_report for category death:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4983
           1       0.56      0.53      0.55       261

   micro avg       0.96      0.96      0.96      5244
   macro avg       0.77      0.76      0.76    

### 9. Export your model as a pickle file

In [9]:
with open('classification.pickle', 'wb') as f:
    pickle.dump(pipeline, f)

(19662,)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.